# Bisection Method

In [ ]:
using Plots

These settings are good default plotting parameters:

In [ ]:
default(lw=2,markersize = 6,
    xtickfont=font(12), ytickfont=font(12), 
    guidefont=font(14), legendfont=font(12),titlefont=font(12))

In [ ]:
"""
    bisection(f, a, b, n_max, tol; verbose = true)

Find a root of function `f` in the interval [a,b] using the bisection method.

# Arguments
- `f`: Function for which to find the root
- `a`: Left endpoint of the initial interval (must satisfy f(a)*f(b) < 0)
- `b`: Right endpoint of the initial interval (must satisfy f(a)*f(b) < 0)
- `n_max`: Maximum number of iterations allowed
- `tol`: Tolerance for convergence (algorithm stops when interval width < 2*tol)
- `verbose`: Optional keyword argument. If true, prints iteration information (default: true)

# Returns
- `p`: Approximate root of the function

# Method
The bisection method works by:
1. Computing the midpoint p = (a+b)/2 of the current interval [a,b]
2. Evaluating f(p) and determining which subinterval [a,p] or [p,b] contains the root
3. Replacing the interval with the subinterval that contains the root
4. Repeating until convergence or maximum iterations reached

# Convergence
The algorithm converges when either:
- f(p) = 0 (exact root found)
- The interval width (b-a)/2 < tol

# Example
```julia
f(x) = x^3 + 4*x^2 - 10
root = bisection(f, 1, 2, 100, 1e-8)
```

# Notes
- Requires that f(a) and f(b) have opposite signs (Intermediate Value Theorem)
- Convergence is guaranteed but may be slow compared to other methods
- Error decreases by factor of 2 each iteration: |p_n - p| ≤ (b-a)/2^n
"""
function bisection(f, a, b, n_max, tol; verbose = true)
    
    converged = false;
    p = 0;
    for i in 1:n_max

        p = 0.5 * (a+b); # compute the midpoint 
        
        # print current iterate information to screen 
        if verbose
            println(" $i: a = $(round(a,digits=8)), b = $(round(b,digits=8)), p = $(round(p,digits=8)), |f(p)|  = $(round(abs(f(p)),digits=8))")
        end

        # determine if the root is in the left or right interval
        if ( f(a) * f(p)<=0)
            b = p; # root is interval [a,p]
        else
            a = p # root is in interval [p,b]
        end
        if(f(p)==0)
            converged = true;
            break
        end
        
        # test for convergence
        if .5*(b-a)< tol
            converged = true;
            break
        end
    end
    
    if !converged
        println("ERROR: Did not converge after $n_max iterations")
    end

    return p # return midpoint guess
    
end

## Example 1
Find a root of $x^3 + 4x^2 - 10$ in $[1,2]$.

In [ ]:
f(x) = x^3 + 4 * x^2 - 10;
a = 1;
b = 2;
n_max = 100;
tol = 1e-8;

bisection(f, a, b, n_max, tol);

## Bisection Method Iteration Visualization

The following cell performs the bisection method for 10 iterations to find a root of the function `f(x) = x^3 + 4x^2 - 10` in the interval [1, 2]. It records the endpoints and midpoint of the interval at each iteration. Then, it creates an animation that visually shows how the interval containing the root shrinks with each iteration, illustrating the convergence of the bisection method.

In [ ]:
n_max = 10;
a_vals = zeros(n_max);
b_vals = zeros(n_max);
p_vals = zeros(n_max);
a = 1;
b = 2;
p = 0;
# this stores the endpoints and the midpoint at each iteration in the arrays
for i in 1:n_max
    a_vals[i] = a;
    b_vals[i] = b;
    p = 0.5 *  (a+b);
    p_vals[i] = p;
    println(" $i: a = $(round(a,digits=8)), b = $(round(b,digits=8)), p = $(round(p,digits=8)), |f(p)|  = $(round(abs(f(p)),digits=8))")
    if ( f(a) * f(p)<=0)
        b = p;
    else
        a = p
    end    
end

# visualize the bisection method
anim = @animate for i=1:n_max
    xx = LinRange(1,2,100);
    ff = f.(xx);
    plot(xx, ff, label="f(x)")
    plot!([a_vals[i],b_vals[i]], [0,0], label="",lw=6,ls=:dot)
    plot!([p_vals[i],p_vals[i]],[0,f(p_vals[i])],label="",lw=4)
    xlims!(1,2)
    ylims!(f(1), f(2))
    
    xlabel!("x");
    ylabel!("y")
    title!(string("n = $i"))
end

In [ ]:
# animate
gif(anim,  fps = 1)

## Conservative Nature of the Error Estimator
The number of iterations the theorem predicts that to get 
$$
|p_n -p|< {\rm TOL},
$$
it is sufficient to take $n$ large enough such that
$$
\frac{b-a}{2^n} < {\rm TOL}
$$

will be needed to satisfy a particular absolute error is often much higher than needed.

Suppose, for the above problem, we want an absolute error $<10^{-3}$.  


In [ ]:
p_exact = 1.3652300134141; # found elsewhere
a = 1;
b = 2;
TOL = 10^-3;

@show n_max= ceil(Int, log2((b-a)/TOL));

p = 0;
for i in 1:n_max
    p = 0.5 *  (a+b);
    if ( f(a) * f(p)<=0)
        b = p;
    else
        a = p
    end
    println("$i: |p - p_exact| = $(round(abs(p - p_exact),digits=12))")
end


Note that we had an answer with error $\approx 10^{-6}$ after 9 iterations, even though the theorem said we needed 10.